###Objetivo:
O problema em questão envolve a análise de dados históricos recentes relacionados aos cadastros de clientes no sistema de pagamentos instantâneos Pix, com o objetivo de identificar padrões, oportunidades e desafios para aumentar a base de usuários cadastrados. A partir dessa análise, é possível desenvolver estratégias eficazes que impulsionem a adoção do Pix, tanto por parte de clientes individuais quanto de empresas, ampliando assim a participação no mercado de pagamentos digitais.


Sendo assim desenvolvi questionamentos estratégicos, direcionados ao objetivo e servindo como um guia para desenvolvimento deste projeto MVP. 

1. Qual tipo de chave é mais utilizado pelos usuários?
2. Qual instituição que possui o maior número de chaves pix ativas?
3. Como o número total de chaves Pix ativas tem evoluído mês a mês?
4. Qual é a proporção de chaves Pix cadastradas por Pessoa Física versus Pessoa Jurídica?



### Coleta:
Os dados foram coletados do portal de dados abertos do Banco Central do Brasil, exportados diretamente no formato CSV, contendo 84.681 registros. Os dados brutos foram persistidos no DBFS (Databricks File System) para processamento e análise.

###Importação dos dados de Estoque de Chaves Pix:
O código realiza a leitura de um arquivo CSV armazenado no DBFS, utilizando o PySpark. Durante a importação, assume-se que a primeira linha do arquivo contém os cabeçalhos das colunas. Os dados são então carregados em um DataFrame do Spark, que permite manipulação distribuída dos dados em um ambiente clusterizado. Por fim é exibido as 10 primeiras linhas do DataFrame, garantindo que a carga foi realizada corretamente antes de prosseguir com transformações.

In [0]:
dados_brutos_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/pedroliradecastro@gmail.com/Estoque_de_Chaves_Pix_por_Participante.csv")

dados_brutos_df.show(10)

+----------+--------+--------------------+---------------+---------+---------+
|      Data|    ISPB|                Nome|NaturezaUsuario|TipoChave|qtdChaves|
+----------+--------+--------------------+---------------+---------+---------+
|2024-04-30|05442029|CASA CREDITO S.A....|             PF|  Celular|       46|
|2024-06-30|07564184| COOP ASCOOB SERTÕES|             PF|  Celular|        6|
|2024-04-30|30723886|      BCO MODAL S.A.|             PF|  Celular|      381|
|2025-01-31|33641877|          EQUIS BANK|             PJ|   e-mail|        4|
|2024-10-31|07097064|COOP SICOOB CONFI...|             PF|Aleatória|     2427|
|2024-12-31|70431630|COOP SICREDI UNIV...|             PF|Aleatória|    31187|
|2024-12-31|92934215|    BANRISUL S.A. IP|             PJ|Aleatória|    58545|
|2025-02-28|08561701|PAGSEGURO INTERNE...|             PJ|   e-mail|   362399|
|2024-05-31|09639338|CECM PRAIA GRANDE...|             PJ|  Celular|        1|
|2024-10-31|05428587|COOP CRESOL OLIVE...|          

###Apagando o database "bronze" para ser recriado: 
Este comando exclui permanentemente o banco de dados e todos os seus objetos internos. 

In [0]:
%sql DROP DATABASE bronze CASCADE

###Recriando o database "bronze":
Este comando cria um novo banco de dados no ambiente Spark SQL (Databricks).

In [0]:
%sql CREATE DATABASE bronze

###Criação da tabela `chaves_pix` a partir do dataframe:
Salva o DataFrame como uma tabela Delta no schema bronze, substituindo completamente qualquer tabela existente com o mesmo nome. O formato Delta garante recursos como transações ACID e versionamento, enquanto o modo overwrite apaga os dados anteriores sem confirmação. Ideal para persistir dados brutos na camada bronze em pipelines ETL.

In [0]:
dados_brutos_df.write.format("delta").mode("overwrite").saveAsTable("bronze.chaves_pix")

###Verificação da estrutura da tabela:
Consulta simples para visualizar as primeiras 10 linhas da tabela.

In [0]:
%sql SELECT * FROM bronze.chaves_pix LIMIT 10;

Data,ISPB,Nome,NaturezaUsuario,TipoChave,qtdChaves
2024-04-30,05442029,CASA CREDITO S.A. SCM,PF,Celular,46
2024-06-30,07564184,COOP ASCOOB SERTÕES,PF,Celular,6
2024-04-30,30723886,BCO MODAL S.A.,PF,Celular,381
2025-01-31,33641877,EQUIS BANK,PJ,e-mail,4
2024-10-31,07097064,COOP SICOOB CONFIANÇA,PF,Aleatória,2427
2024-12-31,70431630,COOP SICREDI UNIVALES MT/RO,PF,Aleatória,31187
2024-12-31,92934215,BANRISUL S.A. IP,PJ,Aleatória,58545
2025-02-28,08561701,PAGSEGURO INTERNET IP S.A.,PJ,e-mail,362399
2024-05-31,09639338,CECM PRAIA GRANDE E REGIãO,PJ,Celular,1
2024-10-31,05428587,COOP CRESOL OLIVEIRA DOS BREJINHOS,PJ,e-mail,1


###Apagando o database "silver" para ser recriado: 
Este comando exclui permanentemente o banco de dados e todos os seus objetos internos. 

In [0]:
%sql DROP DATABASE silver CASCADE

###Recriando o database "silver":
Este comando cria um novo banco de dados no ambiente Spark SQL (Databricks).

In [0]:
%sql CREATE DATABASE silver

###Transformação de Dados: 
Foi realizado o processamento de dados brutos da camada bronze para a camada silver, aplicando limpeza e enriquecimento. Primeiro, foi importado funções essenciais do PySpark (col, year, month, dayofmonth) para manipulação de colunas e datas. Em seguida, carrega os dados da tabela Delta Lake bronze.chaves_pix em um DataFrame (bronze_df).
A transformação principal inclui:
Filtragem de qualidade: Remove registros com valores nulos na coluna qtdChaves, garantindo dados consistentes.
Enriquecimento temporal: Adiciona três novas colunas derivadas da data (ano, mes, dia), extraindo esses componentes para facilitar análises temporais.
O resultado é um DataFrame (silver_df) pronto para ser persistido como uma tabela Delta Lake na camada silver, com dados limpos e estruturados. 


In [0]:
from pyspark.sql.functions import col, dayofmonth, month, year

bronze_df = spark.table("bronze.chaves_pix")

silver_df = bronze_df.filter(col("qtdChaves").isNotNull())

silver_df = silver_df.withColumn("ano", year(col("Data")))
silver_df = silver_df.withColumn("mes", month(col("Data")))
silver_df = silver_df.withColumn("dia", dayofmonth(col("Data")))

###Salvando em uma tabela:
O comando salva o DataFrame silver_df como uma tabela Delta Lake chamada silver.chaves_pix, substituindo completamente qualquer versão existente da tabela. O formato Delta garante recursos avançados como transações seguras (ACID), versionamento de dados e otimizações de desempenho. A tabela é armazenada como uma tabela gerenciada no metastore do Databricks, pronta para próxima etapa do pipeline.

In [0]:
silver_df.write.format("delta").mode("overwrite").saveAsTable("silver.chaves_pix")

###Verificação da estrutura da tabela:
Consulta simples para visualizar as primeiras 10 linhas da tabela.

In [0]:
%sql SELECT * FROM silver.chaves_pix LIMIT 10;

Data,ISPB,Nome,NaturezaUsuario,TipoChave,qtdChaves,ano,mes,dia
2024-04-30,05442029,CASA CREDITO S.A. SCM,PF,Celular,46,2024,4,30
2024-06-30,07564184,COOP ASCOOB SERTÕES,PF,Celular,6,2024,6,30
2024-04-30,30723886,BCO MODAL S.A.,PF,Celular,381,2024,4,30
2025-01-31,33641877,EQUIS BANK,PJ,e-mail,4,2025,1,31
2024-10-31,07097064,COOP SICOOB CONFIANÇA,PF,Aleatória,2427,2024,10,31
2024-12-31,70431630,COOP SICREDI UNIVALES MT/RO,PF,Aleatória,31187,2024,12,31
2024-12-31,92934215,BANRISUL S.A. IP,PJ,Aleatória,58545,2024,12,31
2025-02-28,08561701,PAGSEGURO INTERNET IP S.A.,PJ,e-mail,362399,2025,2,28
2024-05-31,09639338,CECM PRAIA GRANDE E REGIãO,PJ,Celular,1,2024,5,31
2024-10-31,05428587,COOP CRESOL OLIVEIRA DOS BREJINHOS,PJ,e-mail,1,2024,10,31


###Apagando o database "gold" para ser recriado: 
Este comando exclui permanentemente o banco de dados e todos os seus objetos internos. 

In [0]:
%sql DROP DATABASE gold CASCADE

###Recriando o database "gold":
Este comando cria um novo banco de dados no ambiente Spark SQL (Databricks).

In [0]:
%sql CREATE DATABASE gold

###Criação de Modelo Dimensional (Esquema Estrela) na Camada Gold:

O código SQL transforma dados da camada silver em um modelo dimensional na camada gold, composto por uma tabela de fatos e quatro tabelas de dimensão para análise de dados de chaves PIX.

Estrutura Criada:
- Tabela de Fatos:

Armazena os dados transacionais principais: Data, ISPB, NaturezaUsuario, TipoChave e qtdChaves.

- Tabelas de Dimensão:

Tempo: Extrai ano, mês e dia da data para análises temporais.

Instituição: Lista instituições financeiras (ISPB e Nome) sem duplicatas.

Usuário: Cataloga naturezas de usuários distintas.

Tipo de Chave: Relaciona todos os tipos de chaves PIX existentes.

- Como Funciona:

Usei o CREATE OR REPLACE TABLE para sobrescrever as tabelas existentes, DISTINCT nas dimensões para evitar duplicatas.
Os dados são persistidos como Delta Lake (padrão Databricks), permitindo versionamento e otimizações.

- Objetivo:

Estruturar os dados para consultas analíticas eficientes, integração com ferramentas de BI escalabilidade do modelo.

In [0]:
%sql
CREATE OR REPLACE TABLE gold.fatos_chaves_pix AS
SELECT 
  Data,
  ISPB,
  NaturezaUsuario,
  TipoChave,
  qtdChaves
FROM silver.chaves_pix;


CREATE OR REPLACE TABLE gold.dimensao_tempo AS
SELECT
    DISTINCT Data,
    YEAR(Data) AS ano,
    MONTH(Data) AS mes,
    DAY(Data) AS dia
FROM silver.chaves_pix;


CREATE OR REPLACE TABLE gold.dimensao_instituicao AS
SELECT
    DISTINCT ISPB,
    Nome
FROM silver.chaves_pix;


CREATE OR REPLACE TABLE gold.dimensao_usuario AS
SELECT
    DISTINCT NaturezaUsuario
FROM silver.chaves_pix;


CREATE OR REPLACE TABLE gold.dimensao_tipo_chave AS
SELECT
    DISTINCT TipoChave
FROM silver.chaves_pix;

num_affected_rows,num_inserted_rows


###Verificação da estrutura das tabelas:
Consulta simples para visualizar as primeiras 10 linhas da tabela.

In [0]:
%sql SELECT * FROM gold.fatos_chaves_pix LIMIT 10;

Data,ISPB,NaturezaUsuario,TipoChave,qtdChaves
2024-04-30,05442029,PF,Celular,46
2024-06-30,07564184,PF,Celular,6
2024-04-30,30723886,PF,Celular,381
2025-01-31,33641877,PJ,e-mail,4
2024-10-31,07097064,PF,Aleatória,2427
2024-12-31,70431630,PF,Aleatória,31187
2024-12-31,92934215,PJ,Aleatória,58545
2025-02-28,08561701,PJ,e-mail,362399
2024-05-31,09639338,PJ,Celular,1
2024-10-31,05428587,PJ,e-mail,1


In [0]:
%sql SELECT * FROM gold.dimensao_tempo LIMIT 10;

Data,ano,mes,dia
2024-01-31,2024,1,31
2024-07-31,2024,7,31
2024-12-31,2024,12,31
2025-02-28,2025,2,28
2025-01-31,2025,1,31
2024-05-31,2024,5,31
2024-11-30,2024,11,30
2024-09-30,2024,9,30
2024-04-30,2024,4,30
2024-06-30,2024,6,30


In [0]:
%sql SELECT * FROM gold.dimensao_instituicao LIMIT 10;

ISPB,Nome
05241145,COOP CRESOL GETULIO VARGAS
05745533,COOP CRESOL ITATIBA DO SUL
07122321,CC INTEGRADO - SICOOB INTEGRADO
02931668,SICOOB FLUMINENSE
33923798,BANCO MASTER
92874270,BCO DIGIMAIS S.A.
81099491,COOP SICREDI VALE DO PIQUIRI
00694389,CCLA SICOOB CREDICARU SC/RS
03222753,CREDISIS CREDIARI COOP LTDA.
06174009,CC SICOOB ALIANÇA


In [0]:
%sql SELECT * FROM gold.dimensao_usuario LIMIT 10;

NaturezaUsuario
PF
PJ


In [0]:
%sql SELECT * FROM gold.dimensao_tipo_chave LIMIT 10;

TipoChave
Celular
CNPJ
CPF
Aleatória
e-mail


###Qual tipo de chave é mais utilizado pelos usuários?
Com base no resultado obtido, foi possível identificar que a chave mais utilizada foi a chave aleatória, esta informação é de extrema relevância pois impacta desde a experiência do cliente até a competitividade das instituições financeiras e as políticas regulatórias do Banco Central. Para bancos e fintechs, entender essa preferência permite:

•	Otimizar o cadastro, tornando-o mais ágil e intuitivo, reduzindo abandonos.

•	Reforçar a segurança, direcionando esforços para combater fraudes em chaves mais vulneráveis (como e-mail).



In [0]:
%sql
SELECT 
    TipoChave,
    SUM(qtdChaves) AS total_chaves_utilizadas
FROM 
    gold.fatos_chaves_pix
GROUP BY 
    TipoChave
ORDER BY 
    total_chaves_utilizadas DESC
LIMIT 1;

TipoChave,total_chaves_utilizadas
Aleatória,5.167268687E9


###Qual instituição que possui o maior número de chaves pix ativas?

Como resultado foi visto que NU Pagamentos foi a instuiticao que obteve o maior share de mercado, saber qual instituição financeira lidera em chaves Pix ativas não é apenas uma curiosidade competitiva, mas uma informação estratégica para múltiplos agentes do ecossistema financeiro. Como exemplo:

- Para as instituições financeiras

Benchmarking competitivo: Bancos e fintechs podem comparar sua participação com a líder e ajustar estratégias de atração/retenção de clientes.

Oportunidades de parceria: Instituições menores podem buscar alianças com a líder para ampliar seu alcance.

- Para os usuários

Indicador de confiança: Uma instituição com muitas chaves ativas pode sinalizar maior aceitação no mercado, influenciando escolhas.

Melhores serviços: A concorrência para liderar o ranking pode resultar em benefícios como taxas reduzidas ou funcionalidades exclusivas.

- Para o ecossistema de pagamentos

Atração de investimentos: A líder em chaves Pix se torna um ator central para integrações com e-commerces, aplicativos e outros serviços.

Inovação acelerada: A disputa pela liderança estimula o lançamento de novos produtos (ex.: Pix parcelado, Pix internacional).


In [0]:
%sql
SELECT 
    di.Nome AS nome_instituicao,
    SUM(fc.qtdChaves) AS total_chaves_pix_ativas
FROM 
    gold.fatos_chaves_pix fc
JOIN 
    gold.dimensao_instituicao di
ON 
    fc.ISPB = di.ISPB
GROUP BY 
    di.Nome
ORDER BY 
    total_chaves_pix_ativas DESC
LIMIT 1;

nome_instituicao,total_chaves_pix_ativas
NU PAGAMENTOS - IP,2.17554295E9


###Como o número total de chaves Pix ativas tem evoluído mês a mês?
A evolução mensal do número de chaves Pix ativas é um termômetro estratégico da digitalização financeira no Brasil. Para o Banco Central, revela a eficácia de políticas de inclusão e a necessidade de ajustes regulatórios. Instituições financeiras usam esses dados para direcionar campanhas de captação, otimizar infraestrutura e identificar concorrentes dominantes. Para a sociedade, números robustos indicam democratização do acesso, mas também exigem campanhas de segurança já que o Brasil superou 400 milhões de chaves em 2023, muitas vinculadas a um mesmo usuário. Se o ritmo mensal acelera, sinaliza inovação; se desacelera, aponta maturidade do mercado, demandando estratégias de fidelização. Em resumo, essa métrica não só mede o sucesso do Pix, mas orienta o futuro dos pagamentos digitais no país.

In [0]:
%sql
SELECT 
    YEAR(fc.Data) AS ano,
    MONTH(fc.Data) AS mes,
    SUM(fc.qtdChaves) AS total_chaves_pix_ativas
FROM 
    gold.fatos_chaves_pix fc
GROUP BY 
    YEAR(fc.Data), MONTH(fc.Data)
ORDER BY 
    ano DESC, mes DESC;

ano,mes,total_chaves_pix_ativas
2025,2,8.36096464E8
2025,1,8.28099454E8
2024,12,8.16751696E8
2024,11,8.04946954E8
2024,10,8.05629306E8
2024,9,7.99867417E8
2024,8,7.89063752E8
2024,7,7.78446278E8
2024,6,7.65527393E8
2024,5,7.53568144E8


###Qual é a proporção de chaves Pix cadastradas por Pessoa Física versus Pessoa Jurídica?
Atualmente, PF dominam (95% das chaves), mostrando que o Pix já conquistou usuários comuns, impulsionado por sua praticidade e inclusão financeira. Já a participação de PJ, ainda menor, indica oportunidade de crescimento no mercado, especialmente com funcionalidades como Pix Cobrança.

In [0]:
%sql
SELECT 
    NaturezaUsuario, 
    SUM(qtdChaves) AS total_chaves,
    (SUM(qtdChaves) * 100.0 / (SELECT SUM(qtdChaves) FROM gold.fatos_chaves_pix)) AS proporcao_percentual
FROM 
    gold.fatos_chaves_pix
GROUP BY 
    NaturezaUsuario;

NaturezaUsuario,total_chaves,proporcao_percentual
PF,1.0357059209E10,95.13939906319169
PJ,5.2913443E8,4.860600936808304


###Solução do problema:
Com base nas respostas obtidas para resolver o problema apresentado, foi possível identificar que o Pix se consolidou como um dos principais meios de pagamento instantâneo no Brasil. Atualmente, a grande maioria das chaves ativas está vinculada a pessoas físicas 95%, enquanto apenas 5% estão associadas a pessoas jurídicas. Com mais de 400 milhões de chaves ativas, o crescimento acelerado do sistema exige a adoção de estratégias para evitar sua saturação, como a otimização das chaves existentes e a expansão das funcionalidades do Pix, incorporando novos métodos de pagamento.

O Nubank se destaca como líder em número de chaves registradas, resultado de sua abordagem digital simplificada e de um sólido market share de clientes. Esse cenário pressiona outras instituições financeiras a aprimorarem a experiência do usuário, oferecendo benefícios competitivos e desenvolvendo estratégias eficazes para aumentar suas bases de clientes. 

Para garantir a expansão do Pix, especialmente entre as pessoas jurídicas, algumas ações se mostram essenciais. A educação financeira e a integração do Pix com sistemas empresariais, como ERPs e plataformas de e-commerce, são fundamentais para a adesão de empresas. Além disso, é importante investir em inovações regulatórias, como incentivos fiscais e novas modalidades de pagamento. Outro ponto crucial é o reforço da segurança, com a migração para chaves aleatórias e o combate contínuo a fraudes.

Com a implementação dessas medidas, o Pix não apenas manterá seu crescimento, mas se tornará ainda mais relevante no ecossistema de pagamentos digitais, atendendo de forma eficiente tanto os usuários finais quanto as empresas.

###Autoavaliação:
O desenvolvimento deste projeto de engenharia de dados foi um grande desafio para mim, especialmente porque não possuo experiência prévia com os temas abordados nem com as ferramentas utilizadas. No entanto, acredito que consegui atingir os objetivos propostos e concluir o projeto com êxito, abordando de forma adequada todas as etapas solicitadas e implementando um pipeline de dados completo, utilizando uma tecnologia em nuvem. 

Fiquei extremamente satisfeito com os resultados obtidos, pois as respostas geradas proporcionaram insights valiosos para a tomada de decisões, o que validou o trabalho realizado. Durante o processo, elenquei algumas perguntas que considerei essenciais para compreender melhor o tema, levando em conta tanto o tempo disponível para a conclusão do projeto quanto sua complexidade e extensão.

Concluo minha autoavaliação com a satisfação de ter adquirido novos conhecimentos e me sinto pronto para enfrentar novos desafios e projetos, com uma compreensão mais aprofundada e experiência prática em áreas que antes eram desconhecidas para mim.